In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import jieba
import torch
import pickle
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from ark_nlp.model.ner.span_bert import SpanBert
from ark_nlp.model.ner.span_bert import SpanBertConfig
from ark_nlp.model.ner.span_bert import Dataset
from ark_nlp.model.ner.span_bert import Task
from ark_nlp.model.ner.span_bert import get_default_model_optimizer
from ark_nlp.model.ner.span_bert import Tokenizer

In [2]:
# 目录地址

train_data_path = '../data/source_datasets/CMeEE/CMeEE_train.json'
dev_data_path = '../data/source_datasets/CMeEE/CMeEE_dev.json'
test_data_path = '../data/source_datasets/CMeEE/CMeEE_test.json'

### 一、数据读入与处理

#### 1. 数据读入

In [3]:
train_data_df = pd.read_json(train_data_path)
dev_data_df = pd.read_json(dev_data_path)

In [4]:
train_data_df = train_data_df.rename(columns={'entities': 'label'})
dev_data_df = dev_data_df.rename(columns={'entities': 'label'})

In [5]:
train_data_df = train_data_df.loc[:,['text', 'label']]
train_data_df['label'] = train_data_df['label'].apply(lambda x: str(x))
dev_data_df = dev_data_df.loc[:,['text', 'label']]
dev_data_df['label'] = dev_data_df['label'].apply(lambda x: str(x))

In [6]:
ner_train_dataset = Dataset(train_data_df)
ner_dev_dataset = Dataset(dev_data_df)

#### 2. 词典创建和生成分词器

In [7]:
tokenizer = Tokenizer(vocab='nghuyong/ernie-1.0', max_seq_len=50)

#### 4. ID化

In [8]:
ner_train_dataset.convert_to_ids(tokenizer)
ner_dev_dataset.convert_to_ids(tokenizer)

<br>

### 二、模型构建

#### 1. 模型参数设置

In [9]:
config = SpanBertConfig.from_pretrained('nghuyong/ernie-1.0', 
                                         num_labels=len(ner_train_dataset.cat2id))

#### 2. 模型创建

In [10]:
torch.cuda.empty_cache()

In [11]:
dl_module = SpanBert.from_pretrained('nghuyong/ernie-1.0', 
                                         config=config)

Some weights of the model checkpoint at nghuyong/ernie-1.0 were not used when initializing SpanIndependenceBert: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing SpanIndependenceBert from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SpanIndependenceBert from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SpanIndependenceBert were not initialized from the model checkpoint at nghuyong/ernie-1.0 and are newly initialized: ['classifier.wei

<br>

### 三、任务构建

#### 1. 任务参数和必要部件设定

In [12]:
# 设置运行次数
num_epoches = 1
batch_size = 32

In [13]:
optimizer = get_default_model_optimizer(dl_module)

#### 2. 任务创建

In [14]:
model = Task(dl_module, optimizer, 'ce', cuda_device=0)

#### 3. 训练

In [ ]:
model.fit(ner_train_dataset, 
          ner_dev_dataset,
          lr=3e-5,
          epochs=5, 
          batch_size=batch_size
         )

<br>

### 四、模型验证与保存

#### 1. 模型验证

In [16]:
from ark_nlp.model.ner.span_bert import Predictor

In [17]:
ner_predictor_instance = Predictor(model.module, tokenizer, ner_train_dataset.cat2id)

In [18]:
ner_predictor_instance.predict_one_sample('（5）房室结消融和起搏器植入作为反复发作或难治性心房内折返性心动过速的替代疗法。')

[{'start_idx': 3, 'end_idx': 7, 'type': 'pro', 'entity': '房室结消融'},
 {'start_idx': 9, 'end_idx': 13, 'type': 'pro', 'entity': '起搏器植入'}]

#### 2. 多样本验证

In [ ]:
test_data_df = pd.read_json(test_data_path)
test_data_df = test_data_df.loc[:,['text']]

record_ = []
for text_ in test_data_df['text'].to_list():
    record_.append([text_, ner_predictor_instance.predict_one_sample(text_)])

#### 3. Batch模型验证

In [21]:
test_data_df = pd.read_json(test_data_path)
test_data_df = test_data_df.loc[:,['text']]

ner_test_dataset = Dataset(test_data_df, categories=ner_train_dataset.categories, is_test=True)

<br>

### 五、模型测试报告

In [ ]:
1. 基本功能测试 通过
2. one sample predict 通过
3. batch predict  无batch方法
4. 多样本验证  通过